# Gerando os dados aleatórios

In [2]:
from math import sqrt, exp, log, erf
import numpy as np
import pandas as pd 

#   Generate data samples
#   We resolve dependent parameters strike and underlying price to ratio strike/underlying
class DataGenerator():

    def generate_random_input(self, n):
        spot = np.random.uniform(1, 150, n)
        strike = np.random.uniform(1, 150, n) # K/S
        time = np.random.uniform(0, 600, n)                      # time until expiration in days
        rate = np.random.uniform(0.4, 0.6, n)                   # Annualized risk free rate
        sigma = np.random.uniform(0.0001, 2.0, n)                # Standard Deviation of stock's returns
        dividend = np.random.uniform(0.0, 0.2, n)                # Dividend yield
        return np.matrix([spot,strike, time, rate, sigma, dividend]).transpose()

    def calc_black_scholes(self, x):
        # shorthanded
        ratio_strike_underlying = x[0]
        time = x[1]
        rate = x[2]
        sigma = x[3]
        dividend = x[4]
        
        sigTsquared = sqrt(time / 365) * sigma
        edivT = exp((-dividend * time) / 365)
        ert = exp((-rate * time) / 365)
        d1 = (log(edivT/ratio_strike_underlying) + (rate + .5 * (sigma ** 2)) * time / 365) / sigTsquared
        d2 = d1 - sigTsquared
        Nd1 = (1 + erf(d1 / sqrt(2))) / 2
        Nd2 = (1 + erf(d2 / sqrt(2))) / 2

        # We are only interested in ratio of premium to underlying price
        ratio_call_premium_underlying = edivT*Nd1 - ratio_strike_underlying * ert * Nd2

        return ratio_call_premium_underlying

    def generate_output(self, input_data):
        return np.apply_along_axis(self.calc_black_scholes, axis=1, arr=input_data)

In [3]:
n_samples = 100
tdg = DataGenerator()
X = tdg.generate_random_input(n_samples) 
dados = pd.DataFrame(data=X, columns=['spot','strike', 'time', 'rate', 'sigma', 'dividend'])



In [4]:
import numpy as np
import scipy.stats as si
import sympy as sy
import sympy.stats as systats

def black_scholes_call_div(S, K, T, r, q, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #q: rate of continuous dividend paying asset 
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

S = 30
K = 40
T = (30/252) #anos
r = 4 #taxa de juros anualizada
q = 9
sigma = 20 #volatilidade 20%
black_scholes_call_div(S, K, T, r, q, sigma)

10.266683332052937